In [4]:
import pandas as pd


## Ingesta de datos (Extracion)

In [5]:
df = pd.read_json("../Dataset/steam_games.json.gz", compression='gzip', lines=True)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se observa que existe una gran cantidad de registros nulos

In [6]:
# se cuentan los valores nulos
df.id.isna().value_counts()

id
True     88312
False    32133
Name: count, dtype: int64

Hay 88312 registros nulos, los cuales procedere a eliminar

## Tratamiento de datos (Transformacion)


In [7]:
# Se eliminan todas las filas donde el 'id 'sea del tipo NaN
df = df.dropna(subset= 'id', axis = 0)
df.shape

(32133, 13)

Se eliminan las columnas que su información no es relevante para el caso particular

In [8]:
df.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [9]:
df.drop(columns= ['publisher', 'app_name', 'url', 'tags', 'specs', 'early_access'], inplace=True)

Se valida los registros nulos

In [10]:
df.isna().sum()

genres          3282
title           2049
release_date    2066
reviews_url        0
price           1377
id                 0
developer       3298
dtype: int64

Se eliminan los valores nulos para optimizar el consumo de datos necesarios para la API

In [11]:
df.dropna(subset=['genres', 'title', 'developer', 'release_date', 'price'], inplace=True)
df.shape

(27462, 7)

In [12]:
df.isna().sum()

genres          0
title           0
release_date    0
reviews_url     0
price           0
id              0
developer       0
dtype: int64

Para darle solucion al primer Endpoint es necesario filtrar la columna 'release_date' solo por año 

In [13]:
import re
def anio(cadena:str) -> str|None:
    """
    La función anio toma la cadena de texto y devuelve un año en formato entero
    o None si la cadena está vacía.
    """
    patron = r'(\d{4})'
    match = re.search(patron, str(cadena))
    if match:
        return match.group(1)
    else:
        return None
df['release_date'] = df['release_date'].apply(anio)

In [14]:
df.head(2)

,genres,title,release_date,reviews_url,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,http://steamcommunity.com/app/643980/reviews/?...,Free To Play,643980.0,Secret Level SRL


 Los registros inciales eran '88312', despues de las transformaciones quedarón '27462' <br> ya estan listos para ser cargados en formato parquet

ultimos arreglos, se eliminan una columna mas que es innecesaria

In [15]:
df.drop(columns=['reviews_url'], inplace=True)

Se modifica la columna price, 'Free to Play' pasa a '0' tipo float

In [16]:
def convertir_a_float(precio:str) -> float:
    try:
        return float(precio)
    except (ValueError, TypeError):
        return 0.0

In [17]:
df['price'] = df['price'].apply(convertir_a_float)

In [18]:
df.head(3)

,genres,title,release_date,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,0.00,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,0.00,670290.0,Poolians.com


Para normalizar las columnas se renombra la columna 'id'

In [19]:
df.rename(columns={'id':'item_id'}, inplace=True)
df.head(4)

,genres,title,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,0.00,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,0.00,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017,0.99,767400.0,彼岸领域


In [20]:
# la lista de generos se acomoda en filas unicas para cada tipo
df = df.explode('genres')
df

,genres,title,release_date,price,item_id,developer
88310,Action,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Casual,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Indie,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Simulation,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Strategy,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
...,...,...,...,...,...,...
120442,Indie,Russian Roads,2018,1.99,610660.0,Laush Dmitriy Sergeevich
120442,Racing,Russian Roads,2018,1.99,610660.0,Laush Dmitriy Sergeevich
120442,Simulation,Russian Roads,2018,1.99,610660.0,Laush Dmitriy Sergeevich
120443,Casual,EXIT 2 - Directions,2017,4.99,658870.0,"xropi,stev3ns"


Se exportan en formato parquet

In [22]:
df.to_parquet('../Dataset/steam_games.parquet')